In [130]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
import re
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [108]:
# # Define the CNN model
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(32 * 16 * 16, 256)
#         self.fc2 = nn.Linear(256, 1)

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = x.view(-1, 32 * 16 * 16)
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x


In [109]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(64 * 8 * 8, 512)
#         self.fc2 = nn.Linear(512, 256)
#         self.fc3 = nn.Linear(256, 1)

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = self.pool(torch.relu(self.conv3(x)))
#         x = x.view(-1, 64 * 8 * 8)
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


In [110]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(128 * 4 * 4, 512)
#         self.fc2 = nn.Linear(512, 256)
#         self.fc3 = nn.Linear(256, 128)
#         self.fc4 = nn.Linear(128, 1)

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = self.pool(torch.relu(self.conv3(x)))
#         x = self.pool(torch.relu(self.conv4(x)))
#         x = x.view(-1, 128 * 4 * 4)
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.relu(self.fc3(x))
#         x = self.fc4(x)
#         return x

In [52]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = self.pool(torch.relu(self.conv5(x)))
        x = x.view(-1, 256 * 2 * 2)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [93]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(1024 * 1 * 1, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = self.pool(torch.relu(self.conv5(x)))
        x = self.pool(torch.relu(self.conv6(x)))
        x = x.view(-1, 1024 * 1 * 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [70]:
# Create training dataset
# Set up the data directories
data_dir = 'Test_Images'
train_dir = os.path.join(data_dir, 'Train')

# Define a function to get the labels from the image filenames
def get_label(filename):
    match = re.search(r'\d+\.?\d*', filename)
    if match:
        return float(match.group())
    else:
        return None

# Define a list to store the image filenames and labels
train_data = []

# Iterate over the training images and add them to the list
for filename in os.listdir(train_dir):
    label = get_label(filename)
    if label is not None:
        train_data.append([os.path.join(train_dir, filename), label])

# Convert the list to a dataframe
train_df = pd.DataFrame(train_data, columns=['filename', 'label'])

# Save the dataframe to a CSV file
train_df.to_csv(os.path.join(data_dir, 'train.csv'), index=False)

In [71]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = self._load_data(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path, label = self.data[index]
        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _load_data(self, csv_file):
    # Load and preprocess data from the CSV file
    # Example: Assuming the CSV file has two columns representing image paths and labels
        data = []
        with open(csv_file, 'r') as file:
            lines = file.readlines()
            for line in lines[1:]:  # Skip the header row
                # Split the line by comma or any other appropriate delimiter
                image_path, label = line.strip().split(',')
                data.append((image_path, float(label)))  # Parse the label as float
        return data

# # Access individual samples
# image, label = dataset[0]
# print(image.shape, label)

In [114]:
# psum    = torch.tensor([0.0, 0.0, 0.0])
# psum_sq = torch.tensor([0.0, 0.0, 0.0])

# # loop through images
# # for inputs in train_data:#DataLoader(train_data, batch_size=batch_size, shuffle=True):
# for i, (images, labels) in enumerate(train_loader):
#     psum    += images.sum(axis        = [0, 2, 3])
#     psum_sq += (images ** 2).sum(axis = [0, 2, 3])
    

# # Final Calculation
# # pixel count
# image_size = 256
# count = 870 * image_size * image_size

# # mean and std
# total_mean = psum / count
# total_var  = (psum_sq / count) - (total_mean ** 2)
# total_std  = torch.sqrt(total_var)

# # output
# print('mean: '  + str(total_mean))
# print('std:  '  + str(total_std))

In [122]:
input_image = Image.open('Test_Images\Train\drop_s41_v5_r0.5_str6_pos0_cam150.png')  # Replace 'input_image.jpg' with the path to your input image file
transform = ToTensor()
tensor = transform(input_image)
# Calculate mean and standard deviation
mean = torch.mean(tensor)
std = torch.std(tensor)

# Normalize the data
normalizer = Normalize(mean=[mean.item(), mean.item(), mean.item()], std=[std.item(), std.item(), std.item()])
# Set up the data
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean= [0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])               # Need to be adjusted, acording to dataset
])


In [123]:
# Example usage:
csv_file = 'Test_Images/train.csv'  # Replace with the path to your CSV file
image_folder = 'Test_Images/train'  # Replace with the path to your image folder


# Split the dataset into training and validation sets
dataset = CustomImageDataset(csv_file, image_folder, transform=transform)
val_split = 0.2  # Percentage of data for validation
val_size = int(val_split * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


# Create data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [128]:
# Define the loss function and optimizer
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=1, bias=True)
)

In [131]:
# num_epochs = 30
# best_val_loss = float('inf')
# patience = 5  # Number of epochs to wait for improvement
# counter = 0  # Counter to track the number of epochs without improvement

# for epoch in range(num_epochs):
#     running_loss = 0.0
    
#     # Training phase
#     model.train()
#     for i, (inputs, labels) in enumerate(train_loader, 0):
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         labels = labels.unsqueeze(1)
#         loss = criterion(outputs, labels.float())
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 100 == 99:
#             print('[%d, %5d] loss: %.6f' % (epoch + 1, i + 1, running_loss / 100))
#             running_loss = 0.0
            
#     # Validation phase
#     model.eval()
#     with torch.no_grad():
#         val_loss = 0.0
#         val_samples = 0
#         for inputs, labels in val_loader:
#             outputs = model(inputs)
#             labels = labels.unsqueeze(1)
#             loss = criterion(outputs, labels.float())
#             val_loss += loss.item() * inputs.size(0)
#             val_samples += inputs.size(0)
        
#         average_val_loss = val_loss / val_samples
#         print('Epoch %d - Validation Loss: %.6f' % (epoch + 1, average_val_loss))
        
#         # Check if validation loss improved
#         if average_val_loss < best_val_loss:
#             best_val_loss = average_val_loss
#             counter = 0
#             torch.save(model, 'trained_model')  # Save the best model
#         else:
#             counter += 1
#             if counter >= patience:
#                 print('No improvement in validation loss. Early stopping.')
#                 break
        
#     print('Epoch %d completed.' % (epoch + 1))

KeyboardInterrupt: 

In [138]:
num_epochs = 30
best_val_loss = float('inf')
patience = 5  # Number of epochs to wait for improvement
counter = 0  # Counter to track the number of epochs without improvement

for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Training phase
    model.train()
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.unsqueeze(1)
        loss = criterion(outputs, labels.float())

        # L1 regularization
        l1_lambda = 0.01  # Adjust the L1 regularization strength
        l1_regularization = torch.tensor(0.)
        for param in model.parameters():
            l1_regularization += torch.norm(param, 1)
        loss += l1_lambda * l1_regularization

        # L2 regularization
        l2_lambda = 0.01  # Adjust the L2 regularization strength
        l2_regularization = torch.tensor(0.)
        for param in model.parameters():
            l2_regularization += torch.norm(param, 2)
        loss += l2_lambda * l2_regularization

        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.6f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_samples = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels.float())
            val_loss += loss.item() * inputs.size(0)
            val_samples += inputs.size(0)
        
        average_val_loss = val_loss / val_samples
        # print('Epoch %d - Validation Loss: %.6f' % (epoch + 1, average_val_loss))
        # Print the running loss and validation loss at the end of the epoch
        print('Epoch %d - Training Loss: %.6f - Validation Loss: %.6f' % (epoch + 1, running_loss / len(train_loader), average_val_loss))

        
        # Check if validation loss improved
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            counter = 0
            torch.save(model, 'trained_model')  # Save the best model
        else:
            counter += 1
            if counter >= patience:
                print('No improvement in validation loss. Early stopping.')
                break
        
    print('Epoch %d completed.' % (epoch + 1))

Epoch 1 - Training Loss: 190.603422 - Validation Loss: 3.509931
Epoch 1 completed.


KeyboardInterrupt: 

In [67]:
model = torch.load('trained_model')  # Replace 'trained_model.pth' with the path to your trained model file

# Load and preprocess the input image
input_image = Image.open('Test_Images\Train\drop_s41_v5_r0.5_str6_pos0_cam150.png')  # Replace 'input_image.jpg' with the path to your input image file
input_image = input_image.convert('RGB')
input_tensor = transform(input_image)

# input_tensor = torchvision.transforms.ToTensor()(input_image).unsqueeze(0)  # Preprocess the image and add a batch dimension
input_tensor.shape

torch.Size([3, 64, 64])

In [68]:
# Set the model in evaluation mode
model.eval()

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)

# Convert the output to a readable format
predicted_value = output.item()

# Print the predicted value
print('Predicted value:', predicted_value)

Predicted value: 42.47267532348633
